## Rev 79 AI Assitant 

In [2]:
#pip install dependecies 
!pip install langchain
!pip install openai
!pip install --upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib
!pip install pinecone-client
!pip install gradio


[notice] A new release of pip available: 22.3.1 -> 23.0.1
[notice] To update, run: pip3 install --upgrade pip

[notice] A new release of pip available: 22.3.1 -> 23.0.1
[notice] To update, run: pip3 install --upgrade pip


In [33]:
# Playing with the new Open AI API : gpt-3.5-turbo
#importing dependencies
import langchain
import openai
from langchain.chat_models import ChatOpenAI
from langchain import LLMChain
from langchain import PromptTemplate
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    AIMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage
)
from langchain.document_loaders import GoogleDriveLoader
import os
from langchain.vectorstores import Chroma, Pinecone
from langchain.embeddings.openai import OpenAIEmbeddings
import pinecone
from langchain.chains.question_answering import load_qa_chain

In [28]:
#using a document loader to connected to a Google Drive folder. I have local credentials file to my Google Drive account.
loader = GoogleDriveLoader(folder_id="1xuVnnIqQCCM8D8gGMPolvx7_mfKJCvbz")                       
docs = loader.load()

In [29]:
#text needs to be split because the Open AI has "token" limits so I use the RecursiveCharacterTextSplitter to split the doc into multiple docs. 
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(docs)

In [ ]:
OPENAI_API_KEY = os.environ["OPENAI_API_KEY"]
PINECONE_API_KEY = os.environ["PINECONE_API_KEY"]
PINECONE_API_ENV = os.environ["PINECONE_API_ENV"]

In [31]:
#Use OpenAI's embedding
embeddings = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY)

In [34]:
# initialize pinecone
pinecone.init(
    api_key=PINECONE_API_KEY,  # find at app.pinecone.io
    environment=PINECONE_API_ENV  # next to api key in console
)
index_name = "langchain2"

In [32]:
#store documentation in Pinecone, a vector database
docsearch = Pinecone.from_texts([t.page_content for t in texts], embeddings, index_name=index_name)

In [35]:
#Initializing gpt-3.5-turbo and llm for question and answer chaining. 
chat = ChatOpenAI(openai_api_key=OPENAI_API_KEY)
chain = load_qa_chain(chat, chain_type="stuff")


In [36]:
#Example query 
query = "What is the difference between program and project?"
docs = docsearch.similarity_search(query, include_metadata=True)

In [37]:
#run the chain to get an answer
chain.run(input_documents=docs, question=query)


'A program is a long-term engagement in an endeavor that may involve multiple projects or planning phases, while a project is a defined plan to achieve specific objectives within a timeframe, sometimes with a specified budget. Programs typically encompass multiple projects, while projects are more specific and have a shorter duration.'

In [38]:
#create a function in order to use it with gradio 
def answer_question(text):
    answer = str(chain.run(input_documents=docs, question=text))
    return answer

In [39]:
import gradio as gr

# create a Gradio interface that accepts the input text and displays the sentiment analysis result
input_text = gr.inputs.Textbox(label="Question")
output_text = gr.outputs.Textbox(label="Answer")

demo = gr.Interface(fn=answer_question, inputs=input_text, outputs=output_text, title="Rev79 Foundational Concepts", description="Ask a question about Rev79.")

demo.launch() 

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/gradio/inputs.py:27: UserWarning: Usage of gradio.inputs is deprecated, and will not be supported in the future, please import your component from gradio.components
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/gradio/deprecation.py:40: UserWarning: `optional` parameter is deprecated, and it has no effect
  warnings.warn(value)
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/gradio/deprecation.py:40: UserWarning: `numeric` parameter is deprecated, and it has no effect
  warnings.warn(value)
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/gradio/outputs.py:22: UserWarning: Usage of gradio.outputs is deprecated, and will not be supported in the future, please import your components from gradio.components
  warnings.warn(


Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.
